# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U dotenv openai azure-ai-projects

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")

# Optimization
OpenAI 는 다양한 최적화 도구를 제공한다. 이를 통해 성능과 비용을 선택적으로 개선해 나갈 수 있다.

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

## Input Token Caching
흔희 많이 쓰이는 건 input token 캐시이다. 비용 측면에서도 50% 정도 절감할 수 있다.

In [ ]:
from openai.types.chat import ChatCompletionUserMessageParam

# chat completions API 호출
response = client.chat.completions.create(
    model=AZURE_OPENAI_CHAT_DEPLOYMENT,
    messages=[
        ChatCompletionUserMessageParam(
            role="user",
            content="입력되는 패턴의 숫자를 이어서 100개 더 출력해줘.\n" + "\n".join([f"{i}" for i in range(1000)]),
        ),
    ],
)

print(response.usage.model_dump_json(indent=2))

# responses API 호출
previous_response_id = None
for season in ["봄", "여름", "가을", "겨울"]:
    response = client.responses.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT,
        input=f"{season} 계절에 잘 어울리는 옷을 추천해줘.",
        previous_response_id=previous_response_id,
    )
    print(f"--- {season} ---")
    print(response.usage.model_dump_json(indent=2))
    previous_response_id = response.id

## Structed Outputs
출력되는 token 량은 LLM 모델이 non-deterministic 하기에 요청마다 다르다. Output Schema 를 정의하여 응답 format 으로 전달하면 해당 구조에 맞게 generate 한다. 모델에 따라 기대하는 token usage 가 달라질 수 있기에 이점 유념하도록 한다.

In [ ]:
import json
from pydantic import BaseModel

class IntentEvent(BaseModel):
    intent: str

# LLM 모델인 gpt-4.1 을 사용하여 한국어로 된 문장의 의도를 분석하고, structed output 으로 반환받아보자.
# gpt-4.1 은 intent 에 해당하는 token 만 generate 한다.
response = client.beta.chat.completions.parse(
    model="gpt-4.1", # replace with the model deployment name of your gpt-4o 2024-08-06 deployment
    messages=[
        {"role": "system", "content": "너는 대화 의도 분석을 전문으로 하는 AI야."},
        {"role": "user", "content": "Alice와 Bob이 금요일에 과학 박람회에 가기로 했어."},
    ],
    response_format=IntentEvent,
)
print("---- gpt-4.1 ----")
print(f"응답: {response.choices[0].message.parsed}")
print(response.usage.model_dump_json(indent=2))

# LLM 모델인 gpt-5 을 사용하여 한국어로 된 문장의 의도를 분석하고, structed output 으로 반환받아보자.
# gpt-5 은 gpt-4.1 보다 많은 token 을 generate 하고, 후처리로 intent 를 추출한다.
response = client.beta.chat.completions.parse(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "너는 대화 의도 분석을 전문으로 하는 AI야."},
        {"role": "user", "content": "Alice와 Bob이 금요일에 과학 박람회에 가기로 했어."},
    ],
    response_format=IntentEvent,
)
print("---- gpt-5 ----")
print(f"응답: {response.choices[0].message.parsed}")
print(response.usage.model_dump_json(indent=2))

# responses API 를 사용해보자. 동일하지만...
response = client.responses.parse(
    model="gpt-5",
    input=[
        {"role": "system", "content": "너는 대화 의도 분석을 전문으로 하는 AI야."},
        {"role": "user", "content": "Alice와 Bob이 금요일에 과학 박람회에 가기로 했어."},
    ],
    text_format=IntentEvent,
)
print("---- gpt-5 ----")
print(f"응답: {response.output_text}")
print(response.usage.model_dump_json(indent=2))

## Predicted Output
Assistants API 는 Conversational API 이다. 단순 answer 를 생성해내는 것을 넘어 사용자와 모델의 대화를 제어할 수 있다. 여전히 beta 이고 벌써 deprecated 된단다.

In [ ]:
import time

from openai.types.chat import ChatCompletionUserMessageParam, ChatCompletionPredictionContentParam


code = """
In the field of artificial intelligence, particularly in the development of large language models, the ability to generate coherent, contextually relevant, and semantically accurate responses over extended conversations has become not only a benchmark for performance but also a critical factor in determining the practical applicability of these models in real-world scenarios, including customer support automation, academic research assistance, and interactive educational tools.
"""
refactor_prompt = """
Replace the "artificial intelligence" with an "AI".
"""

start_time = time.time()
completion = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    messages=[
        ChatCompletionUserMessageParam(role="user", content=refactor_prompt),
        ChatCompletionUserMessageParam(role="user", content=code),
    ],
    prediction=ChatCompletionPredictionContentParam(type="content", content=code),
)
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")
print(completion.choices[0].message.content)

print(completion.model_dump_json(indent=2))